In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from scraping_helper_functions import cook_chess_soup, get_chess_data, cook_chess_game_soup
from cleaning_helper_functions import player_df_cleanup, win_cleanup

In [2]:
# This is an extremely important cell, very very necessary, DO NOT DELETE!
def lentil(x):
    return len(x)

In [3]:
# URL = "https://www.chess.com/games/search?opening=&openingId=&p1=Magnus+Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page=1"
# page = requests.get(URL)

# soup = BeautifulSoup(page._content, "html.parser")

Scrape Chess.com for all of Magnus's otb chess games

In [4]:
# broth = soup.html.body.find('div', class_ = "base-layout").find('div', class_ = "base-container").main
# veggies = broth.find('div', class_ = "layout-column-one").find("div", class_ = "v5-section").find('div', class_ = "v5-section-content-wide").find_all('div', class_ = "post-preview-list-component")[2]
# noodles = veggies.find('div', class_ = 'master-games-table-responsive').table.tbody
# meats = noodles.find_all('tr')

Now we have a list of table entries in html that contains data about each of Magnus's games (on this page at least)
So, lets loop through this list and extract the important data into a dataframe 

Loop through the different pages and grab all the list elements containing data about Magnus's otb games

In [5]:
html_data = []

# Grab Magnus's games from chess.com up until 2015
for num in range(0, 60):
    URL = f"https://www.chess.com/games/search?opening=&openingId=&p1=Magnus%20Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page%3D3&page={num}"
    html_data.extend(cook_chess_soup(URL))

In [6]:
lentil(html_data)

1500

In [7]:
white_player_list = []
white_player_rating_list = []
black_player_list = []
black_player_rating_list = []
result_list = []
game_length_list = []
year_list = []
opening_name_list = []
opening_list = []
game_url_list = []

for chunk in html_data:
    data = get_chess_data(chunk)

    white_player_list.append(data[0])
    white_player_rating_list.append(data[1])
    black_player_list.append(data[2])
    black_player_rating_list.append(data[3])
    result_list.append(data[4])
    game_length_list.append(data[5])
    year_list.append(data[6])
    opening_name_list.append(data[7])
    opening_list.append(data[8])
    game_url_list.append(data[9])

White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan


Now that we have all the lists of data, build the dataframe

In [13]:
# Get the additional data from the game_ids
from scraping_helper_functions import cook_chess_game_soup

for link in game_url_list:
    game_id = link.split('/')[-1]
    URL = f'https://www.chess.com/analysis/game/master/{game_id}'

    cook_chess_game_soup(URL)
    

IndexError: list index out of range

In [ ]:
games_df = pd.DataFrame()

#There is probably a better way to do this but whatever
games_df['white_player'] = white_player_list
games_df['white_player_rating'] = white_player_rating_list
games_df['black_player'] = black_player_list
games_df['black_player_rating'] = black_player_rating_list
games_df['result'] = result_list
games_df['length'] = game_length_list
games_df['year'] = year_list
games_df['opening_name'] = opening_name_list
games_df['opening_moves'] = opening_list


In [ ]:
games_df.info()

Ok, we have the data. Lets clean it.

In [ ]:
#Lets do the easy things first: cleaning the ratings, year, result, length, and opening
games_df['result'] = games_df['result'].map(lambda x: x.strip())
games_df['length'] = games_df['length'].map(lambda x: x.strip())
games_df['opening_moves'] = games_df['opening_moves'].map(lambda x: x.strip())
games_df['opening_name'] = games_df['opening_name'].map(lambda x: x.strip())

In [ ]:
def rating_to_int(rating):
    if type(rating) != float:
        return int(rating[1:5])
    else:
        return rating

In [ ]:
games_df['white_player_rating'] = games_df['white_player_rating'].map(rating_to_int)
games_df['black_player_rating'] = games_df['black_player_rating'].map(rating_to_int)

In [ ]:
games_df

In [ ]:
games_df.info()

In [ ]:
games_df['length'] = games_df['length'].astype('int64')

In [ ]:
games_df.info()

In [ ]:
games_df['result'].value_counts()

In [ ]:
games_df['opening_name'].value_counts()

In [ ]:
opening = []
opening_variant = []
for game in games_df['opening_name'].map(lambda x: x.split(':')):
    opening.append(game[0])
    try: 
        opening_variant.append(game[1])
    except:
        opening_variant.append(np.NaN)

In [ ]:
games_df['opening'] = opening
games_df['opening_variant'] = opening_variant

games_df.drop(columns = 'opening_name', inplace = True)

In [ ]:
len(games_df['opening'].value_counts())

In [ ]:
games_df

In [ ]:
player_df = games_df[['white_player', 'black_player', 'white_player_rating', 'black_player_rating']]

In [ ]:
player_df

In [ ]:
player_df = player_df.apply(player_df_cleanup, axis = 1, result_type = 'expand', name = 'Magnus Carlsen')

In [ ]:
player_df

In [ ]:
player_df.rename(columns = {0 : 'opponent', 1 : 'player_color', 2 : 'opponent_rating', 3 : 'player_rating'}, inplace = True)
player_df

In [ ]:
games_df = games_df.drop(columns = ['white_player', 'black_player', 'white_player_rating', 'black_player_rating'])

In [ ]:
player_df = player_df.join(games_df)

In [ ]:
win_loss_df = player_df.apply(win_cleanup, axis = 1, result_type = 'expand')

In [ ]:
magnus_df = player_df.join(win_loss_df)

In [ ]:
magnus_df[['result', 0, 'player_color']]

In [ ]:
magnus_df.drop(columns = 'result', inplace = True)
magnus_df.rename(columns = {0: 'result'}, inplace = True)

In [ ]:
magnus_df.to_pickle('./magnus.pkl')